In [9]:
import requests     # 2.18.4
import json         # 2.0.9
import pandas as pd # 0.23.0
import numpy as np
import ast
import time
from urlextract import URLExtract as extract
extractor = extract()

In [2]:
# BFE Abnahme
packages = 'https://ckan.ogdch-abnahme.clients.liip.ch/api/3/action/package_search?fq=organization:(bundesamt-fur-energie-bfe)&rows=500'

# Make the HTTP request
response = requests.get(packages)

# Use the json module to load CKAN's response into a dictionary
response_dict = json.loads(response.content)

# Check the contents of the response
assert response_dict['success'] is True  # make sure if response is OK

# Get a list of all publications and their information (each publication is a dictionary)
data = response_dict['result']['results'].copy()

# get all upper keys
allKeys = []

# store information about each dataset in a frame
df = pd.DataFrame(data)[['id','title','url', 'relations', 'resources']]
df

,id,title,url,relations,resources
0,6cf5d98c-e259-4c40-a54b-a93b539eb79b,{'fr': 'Réseaux thermiques: offre de chaleur o...,https://www.bfe.admin.ch/waerme-und-kaelteangebot,[{'url': 'https://www.geocat.ch/geonetwork/srv...,"[{'protocol': 'WWW:DOWNLOAD:Text (.csv), comma..."
1,70faeac6-d660-492d-bc5b-dfdfa707007e,{'fr': 'Réseaux thermiques: demande concernant...,https://www.bfe.admin.ch/waerme-kaeltenachfrag...,[{'url': 'https://www.geocat.ch/geonetwork/srv...,"[{'protocol': 'WWW:DOWNLOAD:Geopackage (ogc)',..."
2,0a6ef29d-86b6-416f-99c3-5e6d26f1f506,{'fr': 'Situation et domaines attenants confor...,https://www.bfe.admin.ch/konsultationsbereiche...,[{'url': 'https://www.geocat.ch/geonetwork/srv...,"[{'protocol': 'OGC:WMS', 'cache_last_updated':..."
3,46ee8db4-c4e9-436a-ad39-bb67102bffec,{'fr': 'Plan sectoriel des lignes de transport...,https://www.bfe.admin.ch/sp-uebertragungsleitu...,[{'url': 'https://www.geocat.ch/geonetwork/srv...,"[{'protocol': 'WWW:DOWNLOAD:Geopackage (ogc)',..."
4,bb13b62d-6cd1-4602-acc1-bbe2aba197b0,{'fr': 'Réseaux thermiques: demande concernant...,https://www.bfe.admin.ch/waerme-kaeltenachfrag...,[{'url': 'https://www.geocat.ch/geonetwork/srv...,"[{'protocol': 'WWW:DOWNLOAD:Geopackage (ogc)',..."
...,...,...,...,...,...
102,a37cb9ae-50f9-4fa5-a48c-95e2c5f0625b,{'fr': 'Aptitude des toitures à utiliser l’éne...,https://www.bfe.admin.ch/bfe/de/home/versorgun...,{},"[{'byte_size': 0, 'cache_last_updated': None, ..."
103,979f37ed-fcb5-4400-802b-be0bb2833ea4,{'fr': 'Statistiques heddomadaires: production...,http://www.bfe.admin.ch/ogd51,[{'url': 'https://www.bfe.admin.ch/bfe/de/home...,"[{'byte_size': 0, 'cache_last_updated': None, ..."
104,73d41982-f474-409a-a129-418678921128,{'fr': 'Etude sur l'électricité hivernale d'or...,https://www.bfe.admin.ch/bfe/de/home/politik/e...,[{'url': 'https://www.bfe.admin.ch/bfe/de/home...,"[{'byte_size': 5819680686, 'cache_last_updated..."
105,e839fa78-ba22-4772-be16-0843b99b7161,{'fr': 'Résultats de la modélisation PE2050+ :...,https://www.bfe.admin.ch/bfe/de/home/politik/e...,[{'url': 'https://www.uvek-gis.admin.ch/BFE/st...,"[{'byte_size': 0, 'cache_last_updated': None, ..."


In [16]:
def check_url(url):
    print('check_url')
    try:
        print(url)
        print(type(url))
        time.sleep(1)
        # head or get
        
        if url == 'https://bfe.admin.ch/shared-mobility-angebote':
            return True
        
        get = requests.get(url)
        
        if get.status_code < 400:
            return True
        else:
            return False
        
    except requests.exceptions.RequestException as e:
        print(e)
        raise SystemExit(f"{url}: is NOT reachable \nErr: {e}")

In [4]:
def addHTTPS(url):
    adjusted = url.replace("http:", "https:" )
    adjusted = adjusted.replace("www", "https://www" )
    
    return adjusted.replace("https://https://www", "https://www" )

In [5]:
def addHTTP(url):
    adjusted = url.replace("www", "http://www")
    return adjusted.replace("http://http://www", "http://www")

In [6]:
def getURLs(urls):
    
    links = []
    
    for url in urls:
        
        url_modified    = ''
        # only store links which are faulty or need a protocol update (https)
        needsToBeListed = False

        if 'https' in url:
            working = check_url(url)
            
            if not working:
                needsToBeListed = True


        else:
            
            url_modified = addHTTPS(url)
            working = check_url(url_modified)
            
            if working:
                needsToBeListed = True
                
            else:
                # does not change http: urls
                url_modified = addHTTPS(url)
                working = check_url(url)
                
                if not working:
                    url_modified = ''
                    needsToBeListed = True
                    
        
        if not needsToBeListed:
            continue


        links.append([url, url_modified])

    return links

In [7]:
def checkString(string):
    
    print('checkString')
    
    urls = extractor.find_urls(string)
    
    # first remove elements which are not actual links
    urls = [url for url in urls if ('http' in url) or ('www' in url)]
    
    return getURLs(urls)

In [10]:
urls = ['http://www.google.ch', 'https://www.google.ch', 'www.google.ch', 'https://data.geo.admin.ch/ch.bfe.windenergie-geschwindigkeit_h150/windenergie-geschwindigkeit_h150/windenergie-geschwindigkeit_h150_2056.fgdb.zip']
getURLs(urls)

check_url
https://www.google.ch
<class 'str'>
check_url
https://www.google.ch
<class 'str'>
check_url
https://www.google.ch
<class 'str'>
check_url
https://data.geo.admin.ch/ch.bfe.windenergie-geschwindigkeit_h150/windenergie-geschwindigkeit_h150/windenergie-geschwindigkeit_h150_2056.fgdb.zip
<class 'str'>


[['http://www.google.ch', 'https://www.google.ch'],
 ['www.google.ch', 'https://www.google.ch'],
 ['https://data.geo.admin.ch/ch.bfe.windenergie-geschwindigkeit_h150/windenergie-geschwindigkeit_h150/windenergie-geschwindigkeit_h150_2056.fgdb.zip',
  '']]

In [13]:
check_url('https://www.bfe.admin.ch/waerme-kaeltenachfrage-wohnen-und-dl')

check_url
https://www.bfe.admin.ch/waerme-kaeltenachfrage-wohnen-und-dl
<class 'str'>


True

In [17]:
cols = ['url', 'relations']

completeList = []

for index, row in df.iterrows():
    
    print(index)
    dataset_name = row['title']['de']
    id           = row['id']
    
    for col in cols:
  
        att_type = 'dataset'
        att      = col
        val      = row[col]

        liste = []

        if col == 'resources':
            
            att_type = 'resources'
            #change att
            print('nichts')


        else:
            if type(val) == str:
                liste = checkString(val)
            else:
                liste = checkString(str(val))


        if len(liste) == 0:
            continue
        else:
            for elem in liste:
                completeList.append([dataset_name, att_type, att, id, elem[0], elem[1]])


0
checkString
check_url
https://www.bfe.admin.ch/waerme-und-kaelteangebot
<class 'str'>
checkString
check_url
https://www.geocat.ch/geonetwork/srv/ger/catalog.search#/metadata/b6c8d9c6-a2ca-435a-af0a-6a8ac94199fc
<class 'str'>
1
checkString
check_url
https://www.bfe.admin.ch/waerme-kaeltenachfrage-wohnen-und-dl
<class 'str'>
checkString
check_url
https://www.geocat.ch/geonetwork/srv/ger/catalog.search#/metadata/32a8a3cd-d269-4915-b983-54fd979ca486
<class 'str'>
2
checkString
check_url
https://www.bfe.admin.ch/konsultationsbereiche-rohrleitungen
<class 'str'>
checkString
check_url
https://www.geocat.ch/geonetwork/srv/ger/catalog.search#/metadata/9188dcff-dced-43b9-8127-b3104f09c175
<class 'str'>
3
checkString
check_url
https://www.bfe.admin.ch/sp-uebertragungsleitungen
<class 'str'>
checkString
check_url
https://www.geocat.ch/geonetwork/srv/ger/catalog.search#/metadata/e1134feb-55d7-4b44-8e13-125e983b259b
<class 'str'>
4
checkString
check_url
https://www.bfe.admin.ch/waerme-kaeltenachfr

33
checkString
check_url
https://www.bfe.admin.ch/baulinien
<class 'str'>
checkString
check_url
https://www.geocat.ch/geonetwork/srv/ger/catalog.search#/metadata/46417fda-0985-4a8e-bc4e-b1026c1b244b
<class 'str'>
34
checkString
check_url
https://www.bfe.admin.ch/windgeschwindigkeit
<class 'str'>
checkString
check_url
https://www.geocat.ch/geonetwork/srv/ger/catalog.search#/metadata/8d78a2a4-03fa-4d95-8a37-57594f2bf7d4
<class 'str'>
35
checkString
check_url
https://www.bfe.admin.ch/windgeschwindigkeit
<class 'str'>
checkString
check_url
https://www.geocat.ch/geonetwork/srv/ger/catalog.search#/metadata/d5892cf4-38fa-4af7-9ddb-f87dd2ed06cd
<class 'str'>
36
checkString
check_url
https://www.bfe.admin.ch/windgeschwindigkeit
<class 'str'>
checkString
check_url
https://www.geocat.ch/geonetwork/srv/ger/catalog.search#/metadata/bb51c6f0-2be5-4933-9582-90dc613c8c75
<class 'str'>
37
checkString
check_url
https://www.bfe.admin.ch/2000-watt-areale
<class 'str'>
checkString
check_url
https://www.geo

checkString
92
checkString
check_url
https://www.bfe.admin.ch/ogd37
<class 'str'>
checkString
93
checkString
check_url
https://www.bfe.admin.ch/ogd38
<class 'str'>
checkString
94
checkString
check_url
https://www.bfe.admin.ch/ogd39
<class 'str'>
checkString
95
checkString
check_url
https://www.bfe.admin.ch/ogd42
<class 'str'>
checkString
96
checkString
check_url
https://www.bfe.admin.ch/ogd43
<class 'str'>
checkString
97
checkString
check_url
https://www.bfe.admin.ch/ogd44
<class 'str'>
checkString
98
checkString
check_url
https://www.bfe.admin.ch/ogd45
<class 'str'>
checkString
99
checkString
check_url
https://www.bfe.admin.ch/ogd46
<class 'str'>
checkString
100
checkString
check_url
https://www.bfe.admin.ch/geoinformation/05061/06360/index.html?lang=de
<class 'str'>
check_url
http://www.bfe.admin.ch/geoinformation/05061/06360/index.html?lang=de
<class 'str'>
checkString
check_url
https://map.geo.admin.ch/?lang=de&topic=energie&bgLayer=ch.swisstopo.pixelkarte-grau&catalogNodes=2419,24

In [18]:
completeList

[['Potenzial der Seen und Flüsse für Wärmeentzug und Wärmeeinleitung',
  'dataset',
  'relations',
  'c13ffb0b-563b-4380-a538-95afb67fc09d',
  'www.bfe.admin.ch/potenzial-waermenutzung-gewaesser',
  'https://www.bfe.admin.ch/potenzial-waermenutzung-gewaesser'],
 ['Kennzahlen Shared Mobility',
  'dataset',
  'url',
  '75b319b5-a7bc-40fc-80e9-579b02e46c02',
  'http://bfe.admin.ch/shared-mobility-angebote',
  'https://bfe.admin.ch/shared-mobility-angebote'],
 ['Kennzahlen Shared Mobility',
  'dataset',
  'relations',
  '75b319b5-a7bc-40fc-80e9-579b02e46c02',
  'www.sharedmobility.ch/info',
  'https://www.sharedmobility.ch/info'],
 ['energiedashboard.ch: Stromverbrauch Prognose SDSC',
  'dataset',
  'url',
  '1bcdc4fe-3e5d-42c2-bcb7-446595653265',
  'www.bfe.admin.ch/digitalisierung',
  'https://www.bfe.admin.ch/digitalisierung'],
 ['Einmalvergütung für Photovoltaikanlagen',
  'dataset',
  'url',
  '5b4852e4-60a7-45f2-bd37-f3d1f16763ed',
  'www.bfe.admin.ch/eiv',
  'https://www.bfe.admin.c

In [21]:
f = pd.DataFrame(completeList)
f.columns=['title', 'type', 'attribute', 'id', 'old_link', 'new_link']
f

,title,type,attribute,id,old_link,new_link
0,Potenzial der Seen und Flüsse für Wärmeentzug ...,dataset,relations,c13ffb0b-563b-4380-a538-95afb67fc09d,www.bfe.admin.ch/potenzial-waermenutzung-gewae...,https://www.bfe.admin.ch/potenzial-waermenutzu...
1,Kennzahlen Shared Mobility,dataset,url,75b319b5-a7bc-40fc-80e9-579b02e46c02,http://bfe.admin.ch/shared-mobility-angebote,https://bfe.admin.ch/shared-mobility-angebote
2,Kennzahlen Shared Mobility,dataset,relations,75b319b5-a7bc-40fc-80e9-579b02e46c02,www.sharedmobility.ch/info,https://www.sharedmobility.ch/info
3,energiedashboard.ch: Stromverbrauch Prognose SDSC,dataset,url,1bcdc4fe-3e5d-42c2-bcb7-446595653265,www.bfe.admin.ch/digitalisierung,https://www.bfe.admin.ch/digitalisierung
4,Einmalvergütung für Photovoltaikanlagen,dataset,url,5b4852e4-60a7-45f2-bd37-f3d1f16763ed,www.bfe.admin.ch/eiv,https://www.bfe.admin.ch/eiv
5,"Füllungsgrad der Speicherseen, Sonntag 24h, Wo...",dataset,url,9eeece7f-56b1-4665-9696-9f07d36dc764,http://www.bfe.admin.ch/ogd17,https://www.bfe.admin.ch/ogd17
6,"Referenz-Marktpreise gemäss Art. 15, EnFV",dataset,url,9a8e7a2c-2404-4a1e-a1e9-d71c178b04d1,http://www.bfe.admin.ch/kev,https://www.bfe.admin.ch/kev
7,Licht-Markt Schweiz,dataset,url,5aded67c-e8d9-401b-b976-ebb558fb8d19,http://www.bfe.admin.ch/ogd25,https://www.bfe.admin.ch/ogd25
8,Angebote der Innovationsförderung im Energiebe...,dataset,url,44f9dd93-6bd0-4d56-8650-e472b44b72b0,http://www.bfe.admin.ch/ogd4,https://www.bfe.admin.ch/ogd4
9,Analyse des schweizerischen Energieverbrauchs ...,dataset,url,cd67848c-9051-44e1-9e72-11c153b46abb,http://www.bfe.admin.ch/ogd3,https://www.bfe.admin.ch/ogd3


In [25]:
f[f.id == 'fdf08159-b7fd-44ba-aaf4-668154f41a43']['old_link']

38    http://www.bfe.admin.ch/geoinformation/05061/0...
Name: old_link, dtype: object

In [22]:
f.to_csv('url_and_relations.csv', index=False)

In [ ]:
check_url('https://www.bfe.admin.ch/waerme-und-kaelteangebot')

In [65]:
for i in range(10):
    check_url('https://www.bfe.admin.ch/waerme-und-kaelteangebot')

check_url
https://www.bfe.admin.ch/waerme-und-kaelteangebot
<class 'str'>


AttributeError: 'tuple' object has no attribute 'tb_frame'

In [40]:
requests.get('https://www.bfe.admin.ch/waerme-und-kaelteangebot')

<Response [200]>

In [4]:
import httplib2
c = httplib2.HTTPSConnection('www.example.com')
c.request("HEAD", '')
if c.getresponse().status == 200:
    print('web site exists')

AttributeError: module 'httplib2' has no attribute 'HTTPSConnection'

In [5]:
import http.client
http.client.HTTPConnection('www.python.org')

In [6]:
http.client.HTTPSConnection('www.python.org')

In [10]:
import httplib2
c = http.client.HTTPConnection('www.bfe.admin.ch/waerme-und-kaelteangebot')
c.request("HEAD", '')
if c.getresponse().status == 200:
    print('web site exists')

gaierror: [Errno -2] Name or service not known

In [12]:
import urllib.request

print(urllib.request.urlopen('https://www.bfe.admin.ch/waerme-und-kaelteangebot').getcode())

ConnectionResetError: [Errno 104] Connection reset by peer

In [83]:
urls = ['https://www.bfe.admin.ch/waerme-und-kaelteangebot']

In [82]:
urls

['http://www.google.ch',
 'https://www.google.ch',
 'www.google.ch',
 'https://data.geo.admin.ch/ch.bfe.windenergie-geschwindigkeit_h150/windenergie-geschwindigkeit_h150/windenergie-geschwindigkeit_h150_2056.fgdb.zip']

In [20]:
import time

In [92]:
for url in urls:
    time.sleep(3)
    r = requests.head(url, headers={"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"})
    print(r.status_code)

ConnectionError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))